<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/Phonetic_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phonetic description practice application

In [ ]:
%%capture
!pip install gradio

In [ ]:
https://raw.githubusercontent.com/MK316/Myapps/refs/heads/main/data/Phonetic-description.csv

In [ ]:
import random
import gradio as gr
import pandas as pd

# Load data from the provided URL
def load_data():
    url = "https://raw.githubusercontent.com/MK316/Myapps/refs/heads/main/data/Phonetic-description.csv"
    try:
        data = pd.read_csv(url, encoding='utf-8')  # Ensure correct encoding
        return data
    except Exception as e:
        raise ValueError(f"Error loading data: {e}")

# Load the data into a DataFrame
df = load_data()

# Function to generate a random question based on user selections
def generate_question(segment_type, description_type):
    segment = "Consonant" if segment_type == "Consonant" else "Vowel"
    description_field = "Full_description" if description_type == "Full_description" else "Casual_description"

    filtered_df = df[df["Segment"] == segment]  # Filter based on consonant/vowel selection

    if filtered_df.empty:
        return "No data available", ""  # Handle empty filter case

    random_row = filtered_df.sample(1).iloc[0]  # Pick a random row
    description = random_row[description_field]  # Select either 'Full_description' or 'Casual_description'
    correct_ipa = random_row["IPA"]  # The correct IPA symbol
    return description, correct_ipa

# Function to check the user's answer
def submit_answer(user_ipa, correct_ipa, score, trials):
    trials += 1
    if user_ipa.strip() == correct_ipa:
        score += 1
        return "Correct!", score, trials
    else:
        return "Try again", score, trials

# Function to quit and show results
def quit_quiz(score, trials):
    return f"Your final score is {score}/{trials}."

# Gradio interface
def gradio_app():
    with gr.Blocks() as app:
        # Radio buttons to select Consonant or Vowel
        segment_type = gr.Radio(choices=["Consonant", "Vowel"], label="Select Consonant or Vowel", value="Consonant")
        # Radio buttons to select description type (Full or Casual)
        description_type = gr.Radio(choices=["Full_description", "Casual_description"], label="Select Description Type", value="Full_description")

        # Start button placed after the selections
        start_button = gr.Button("Show a phonetic description")

        description_output = gr.Textbox(label="Description", interactive=False)
        ipa_input = gr.Textbox(label="Enter IPA", placeholder="Type IPA symbol here")
        submit_button = gr.Button("Submit", visible=False)  # Initially hidden until start is clicked
        result_output = gr.Textbox(label="Result", interactive=False)
        quit_button = gr.Button("Quit")

        # State variables for score and trials
        score_state = gr.State(0)
        trials_state = gr.State(0)
        correct_ipa_state = gr.State("")

        # Start quiz based on selections
        def start(segment_type, description_type):
            description, correct_ipa = generate_question(segment_type, description_type)
            correct_ipa_state.value = correct_ipa  # Save the correct IPA symbol in state
            return description, "", gr.update(visible=True), score_state.value, trials_state.value

        # Submit the answer and check
        def submit(user_ipa):
            result, score, trials = submit_answer(user_ipa, correct_ipa_state.value, score_state.value, trials_state.value)
            score_state.value = score
            trials_state.value = trials
            return result, score, trials

        # Quit and show final results
        def quit():
            return quit_quiz(score_state.value, trials_state.value)

        # Bind actions
        start_button.click(fn=start, inputs=[segment_type, description_type], outputs=[description_output, ipa_input, submit_button, score_state, trials_state])
        submit_button.click(fn=submit, inputs=[ipa_input], outputs=[result_output, score_state, trials_state])
        quit_button.click(fn=quit, outputs=[result_output])

    return app

app = gradio_app()
app.launch()
